In [ ]:
##필요한 모듈들에 대한 설치 진행
!pip install mxnet ##행렬연산을 수행하고, 특히 이미지 처리및 자연어 처리작업에 사용하는 모델
!pip install gluonnlp tdm pandas ##gluonnlp는 nlp처리 모델이며, tdm은 토픽 모델링 관련 도구이다.
!pip install sentencepiece ##텍스트 토큰을 위한 라이브러리이다.
!pip install transformers ##자연어 처리를 위한 파이썬 라이브러리로, 다양한 NLP처리에 사용된다.
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661678 sha256=bdbc6692cacd6c39ed8225d3ac6fc849fc8312855e99f14066a047ec4b04d081
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
  Created wheel for tdm: filename=tdm-0.1.0-py3-none-any.whl size=36753 sha256=1bceed7ad5bde8799834dd22b3a828cff8e51c1af2d620730c1ab11164e2313b
  Stored in directory: /root/.cache/pip/wheels/c6/d8/54/10d04712b2c796b8daf5103a31bbce31af7e1eb6fdf2d2ac77
Successfully built gluonnlp tdm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#kobert설치
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-u7apvtte/kobert-tokenizer_8b4ad05d6c3e48bb9d8e8c3dd90fa709
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-u7apvtte/kobert-tokenizer_8b4ad05d6c3e48bb9d8e8c3dd90fa709
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=83ee53fea7d4d7010f9b3ab35cd0d1c46bf882b28c7c5a68973711308936e60e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ue5oo0m5/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-do84zid1
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-do84z

In [ ]:
# KoBERT import

from kobert_tokenizer import KoBERTTokenizer ##korean Tokenizer을 설치한다.
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
#transformers
from transformers import BertModel ##이미 학습되어있는 BertModel
from transformers import AdamW ##transformer을 위한 옵티마이져
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
# GPU 사용
device = torch.device("cuda:0")

# CPU 사용 시
# device = torch.device("cpu")



In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1') #Bert model에 맞게 토크나이져 역할을 수행한다.
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False) ##텍스트에 대한 임베딩을 수행한다,.
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]') ##Bert model에 사용되는 어휘를 나타낸다.

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋 불러오기
data = pd.read_csv("/content/GPTDataset.csv") ##한글이 포함되어있는 데이터셋이기에 cp949방식으로 encoding을 진행한다.

In [ ]:
data.head() ##데이터 예시

,index,Column1,Label
0,1,복도식 아파트라 많이 덥다.,1
1,2,그러나 아이가 뛰어다니고 놀기에 좋다.,1
2,3,작은 아이라 층간소음이 없는지 아직 소음항의 연락은 없고 윗집 층간소음에 대한 스트...,7
3,4,아파트 맞은편 고4 부지 청아고 개교 예정입니다.,6
4,5,"초중품아(단지와 연결), 아파트 바로 앞 고등학교까지 그리고 중상의 수많은 학원가까...",6


In [ ]:
data.shape

(7289, 3)

In [ ]:
data.loc[(data['Label'] == 8), 'Label'] = 0  # 8 → 0

In [ ]:
data['Label'].unique()

array([1, 7, 6, 5, 4, 0, 3, 2])

In [ ]:
df = pd.DataFrame(data)
df.drop(0, inplace = True)
df

,index,Column1,Label
1,2,그러나 아이가 뛰어다니고 놀기에 좋다.,1
2,3,작은 아이라 층간소음이 없는지 아직 소음항의 연락은 없고 윗집 층간소음에 대한 스트...,7
3,4,아파트 맞은편 고4 부지 청아고 개교 예정입니다.,6
4,5,"초중품아(단지와 연결), 아파트 바로 앞 고등학교까지 그리고 중상의 수많은 학원가까...",6
5,6,미사역도 가깝고 살기 좋아요.,5
...,...,...,...
7284,7285,강북문화정보센터 걸어다닐 수 있어요.,4
7285,7286,공원이있어서 조용하고 평온하게사실분들에게추천해요,1
7286,7287,중앙난방이어서 겨울에 추워요.,1
7287,7288,하지만 바루 뒤에 공원이 있어서 쾌적합니다.,1


In [ ]:
# [Content, 상황] data_list 생성
data_list = []
for ques, label in zip (data['Column1'], data['Label']):
  data = []
  data.append(ques) ##발화문에 해당하는 것을 data에 추가
  data.append(str(label)) #숫자에 해당하는 label을

  data_list.append(data) ##Data List에 한개한개씩 넣는 과정 진행

In [ ]:
print(data)
print(data_list[:10])

['그 대신 여름에는 시원했어요.', '1']
[['복도식 아파트라 많이 덥다.', '1'], ['그러나 아이가 뛰어다니고 놀기에 좋다.', '1'], ['작은 아이라 층간소음이 없는지 아직 소음항의 연락은 없고 윗집 층간소음에 대한 스트레스도 없다단지가 잘 돼있고 역에서 가깝지만힐스테이트 호반써밋처럼 북적거리고 시끄럽지도 않아 애 키우며 살기 참 만족.', '7'], ['아파트 맞은편 고4 부지 청아고 개교 예정입니다.', '6'], ['초중품아(단지와 연결), 아파트 바로 앞 고등학교까지 그리고 중상의 수많은 학원가까지 학세권의 완성이네요', '6'], ['미사역도 가깝고 살기 좋아요.', '5'], ['아직 아이가 어려서 학군은 잘 모르겠는데, 살기는 진짜 좋아요', '1'], ['미사역세권이라서 상가 많고 역이 가까워서 살기 편리합니다.', '5'], ['대신 길가쪽은 많이 시끄러워요.', '7'], ['미사역 가깝고 아이들이 많고 주변 상권도 매우 좋습니다.', '4']]


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)
##20%를 테스트 case로 분리한다.

In [ ]:
print(len(dataset_train), len(dataset_test)) ##총 15499의 train을 위한 dataset과, test dataset이 존재한다.

5831 1458


In [ ]:
# tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False) ##일단 임시로 사용하지는 않는다.

In [ ]:
class BERTSentenceTransform: ##해당 함수의 경우 Bert Model에 넣을 수 있도록 전처리를 진행하는 함수이다.
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True): ##초기화 함수이다.
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length ##입력 데이터의 최대 시퀀스(길이)를 나타낸다.
        self._pad = pad ##입력 sequence를 최대 길이로 padding할 것인지를 물어본다. ##padding을 하여 효율성과 안전성을 높일 수 있다.
        self._pair = pair ##입력이 문자열쌍인지
        self._vocab = vocab ##token과 해당 ID를 포함하는 어휘 객체이다.

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        #해당 함수 사용 진행 기존 Classification 1에 library 형태 사용 변경
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))


In [ ]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
# torch 형식의 dataset을 만들어주면서, 입력 데이터셋의 전처리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 8,   # 클래스 수 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [ ]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

# .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
# 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
test_history.append(test_acc / (batch_id+1))

<ipython-input-29-bd5891c71e9e>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/92 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.254648208618164 train acc 0.046875
epoch 1 train acc 0.531735248447205


  0%|          | 0/92 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2684589624404907 train acc 0.921875
epoch 5 train acc 0.9354619565217391


<ipython-input-29-bd5891c71e9e>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/23 [00:00<?, ?it/s]

epoch 5 test acc 0.8304891304347827


In [ ]:
torch.save(obj=model, f='Trained_Model1.pth')

In [ ]:
temp_model = torch.load('/content/Trained_Model1.pth')

In [ ]:
import csv

# Replace 'your_file.csv' with the actual path to your CSV file.
with open('/content/1_Gracium.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')  # assuming comma is the delimiter
    ReviewArr = list(csv_reader)  # This reads all rows into a list of lists

In [ ]:
ReviewArr.pop(0)
ReviewArr

[['주변에 수영장, 학교 등 진짜 많음'],
 ['녹지가 많아서 산책하기 좋고 운동시설도 주변에 많아서 건강해지는 느낌.'],
 ['살기 진짜 좋은 위치입니다.'],
 ['9호선만 빨리 개통된다면 최고!!'],
 ['대단지 및 학교가 잘배치되어 있어 교육 환경이 좋습니다'],
 ['단지가 쾌적하고 커뮤니티가 잘되어 있어 거주하기 편합니다.'],
 ['상가도 편리하고 아이들 키우기 좋은 동네에요.'],
 ['남쪽동들이 층수가 높아서 가려지는 뒷동은 겨울에는 고층치고는 햇볕이 덜 들어오는 면이 있어요.'],
 ['서울의 주요 회사나 상권과는 거리가 조금 있지만 단지 자체 커뮤니티가 잘 조성되어 있고 조용한 동네라한적한 거 좋아하면 살기 좋은 듯합니다'],
 ['신축이며 관리가 잘 되어 살기 좋는 단지입니다.'],
 ['커뮤니티 센터의 스포츠 프로그램 및 온조대왕 체육관, 도서관이 이용하기 편합니다.'],
 ['너무 살기 좋은 위치인것 같아요.'],
 ['주변에 산책로와 근린공원이 많아서 더욱 쾌적합니다^^'],
 ['장마철 비 많이 와도 비 안맞고지하주차장 통해 이동가능한 점이 넘 좋아요:)'],
 ['주변에 공원, 녹지도 많고 교통도 괜찮은 편.'],
 ['특히 초등자녀 있으면 학교도 가깝고 학원도 밀집해 있어 교육환경이 너무 만족스럽습니다.'],
 ['커뮤니티도 공기도 쟈철도 모두 만족스럽습니다'],
 ['애들 정서와 건강을 위해서 강남구 숲세권 세곡푸르지오로 이사갑니다'],
 ['가보고 반해서 이사갑니다'],
 ['대치동 학원가도 15분컷이고 집앞에 바로 대모산의 대자연이 펼쳐지고 도심속 자연이더군요'],
 ['동네가 유럽 느낌에 고급 리조트같은 아파트단지조경과구성에 반해서 갑니다'],
 ['현재 작은 단지쪽에 사는데매우 매우 만족하고 살고 있네요'],
 ['지하철역 바로 옆이라 출퇴근도 편하고저녁이나 주말에 고덕천으로 운동하기도 좋아요'],
 ['커뮤니티 잘 형성 되어 있고 살기 좋았음.'],
 ['교통편도 좋았음.'],
 ['상가 및 지하철이 지하로 모

In [ ]:
# TEST진행 과정
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    temp_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = temp_model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(0)
            elif np.argmax(logits) == 1:
                test_eval.append(1)
            elif np.argmax(logits) == 2:
                test_eval.append(2)
            elif np.argmax(logits) == 3:
                test_eval.append(3)
            elif np.argmax(logits) == 4:
                test_eval.append(4)
            elif np.argmax(logits) == 5:
                test_eval.append(5)
            elif np.argmax(logits) == 6:
                test_eval.append(6)
            elif np.argmax(logits) == 7:
                test_eval.append(7)
    return test_eval[0]



In [ ]:
for TestText in ReviewArr:
    print(TestText[0])

주변에 수영장, 학교 등 진짜 많음
녹지가 많아서 산책하기 좋고 운동시설도 주변에 많아서 건강해지는 느낌.
살기 진짜 좋은 위치입니다.
9호선만 빨리 개통된다면 최고!!
대단지 및 학교가 잘배치되어 있어 교육 환경이 좋습니다
단지가 쾌적하고 커뮤니티가 잘되어 있어 거주하기 편합니다.
상가도 편리하고 아이들 키우기 좋은 동네에요.
남쪽동들이 층수가 높아서 가려지는 뒷동은 겨울에는 고층치고는 햇볕이 덜 들어오는 면이 있어요.
서울의 주요 회사나 상권과는 거리가 조금 있지만 단지 자체 커뮤니티가 잘 조성되어 있고 조용한 동네라한적한 거 좋아하면 살기 좋은 듯합니다
신축이며 관리가 잘 되어 살기 좋는 단지입니다.
커뮤니티 센터의 스포츠 프로그램 및 온조대왕 체육관, 도서관이 이용하기 편합니다.
너무 살기 좋은 위치인것 같아요.
주변에 산책로와 근린공원이 많아서 더욱 쾌적합니다^^
장마철 비 많이 와도 비 안맞고지하주차장 통해 이동가능한 점이 넘 좋아요:)
주변에 공원, 녹지도 많고 교통도 괜찮은 편.
특히 초등자녀 있으면 학교도 가깝고 학원도 밀집해 있어 교육환경이 너무 만족스럽습니다.
커뮤니티도 공기도 쟈철도 모두 만족스럽습니다
애들 정서와 건강을 위해서 강남구 숲세권 세곡푸르지오로 이사갑니다
가보고 반해서 이사갑니다
대치동 학원가도 15분컷이고 집앞에 바로 대모산의 대자연이 펼쳐지고 도심속 자연이더군요
동네가 유럽 느낌에 고급 리조트같은 아파트단지조경과구성에 반해서 갑니다
현재 작은 단지쪽에 사는데매우 매우 만족하고 살고 있네요
지하철역 바로 옆이라 출퇴근도 편하고저녁이나 주말에 고덕천으로 운동하기도 좋아요
커뮤니티 잘 형성 되어 있고 살기 좋았음.
교통편도 좋았음.
상가 및 지하철이 지하로 모두 연결되어있어서 편리함.
아파트 내 행사가 주말에 종종 열리고 아이들이 많아서 활기있는 느낌.
차를 가지고 양평 등 도시외곽으로 나가기에 아주 좋은 위치입니다.


In [ ]:
# 모든 문장에 대한 classification 진행

with open('/content/10_parkrio.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')  # assuming comma is the delimiter
    ReviewArr = list(csv_reader)  # This reads all rows into a list of lists
ReviewArr.pop(0)

File = open('/content/10_parkrio_Label.csv', 'w',newline='')
data = ['apt_code','Category','Review']
writer = csv.writer(File)
writer.writerow(data)

for TestText in ReviewArr:
    print(TestText[0])
    Ans = predict(TestText[0])
    print(Ans)
    data = ['1Xyb5',Ans,TestText[0]]
    writer.writerow(data)

File.close()

가까운 지하철역올림픽 공원롯데월드몰 등.. 삶의 만족도가 너무 높았던 집입니다
5
올림픽공원, 석촌호수, 한강 모두 가깝고롯데타워도 가까워요~~
5
학군은 말할것도 없고 누가 살던 최고인듯
6
교통도 편하고 주변에 시설이 많아 살기 좋습니다
5
올림픽공원 가깝고 내부 조경및 놀이터가 좋아서 아이들이 살기좋음
1
지하철역하고 가깝고, 유해시설이 없는게 너무좋네요
5
태풍이 오기전이라서 한강뷰가 너무 이쁘네요
1
병원, 지하철, 잠실 쇼핑몰 가까운 편이라 좋았습니다
4
살았던 곳중에 제일 좋네요.
1
역 가깝고 병원 공원 백화점 등등 인프라가 잘 되어있어요.
4
나무 조경을 어디서 한건지...나무들이 더벅머리가 되었어요.
1
단지 학군 교통 최고입니다
6
여자 고등학교가 없어서 아쉽네요
6
윗집만 잘 만나고 화장실 문만 잘 닫는다면 완벽합니다그만 좀 움직여 하...
1
2호선과 가깝고 조용하고 이만큼 살기 좋은 아파트가 있을까 싶네요.
5
그리고 잠실역에 상권까지 이용 할 수 있다는 것도 장점입니다.
5
혼자 사시는 분이면 진심으로 추천드립니다.
1
파크리오 10년동안 거주 하였습니다
1
단지내 조경도 너무 좋고 한강, 올림픽공원을 걸어서 갈 수 있는 것 또한 큰 매력입니다
1
단점으로는 엘베수리와 층간소음, 1층 거주시 베란다 하수구 역류
7
윗집에서 세탁기를 두면 안되는 곳에 두어서 물이 역류할때가 있는데 관리실에서 제대로된 해결방법이 없기에 만약 하수구 역류로 물건 피해를 입으시면 관리실에 피해보상액을 꼭 받으세요
1
집안에서 담배를 피우는 예의없는 행동 등이 있습니다,,
1
초등학교는 두개가 있으나 중학교가 없어서 다른 곳으로 전학가는 아이들도 많습니다
6
주차공간은 넓어서 좋습니다사람들이 만들어 내는 피해 빼고는 정말 강추하고픈 아파트입니다
0
2호선 잠실나루역 / 8호선 몽촌토성역 지척. 단지 내 잠현초, 잠실초, 잠실고 위치. 한강시민공원과 올림픽공원 근접. 단지 전체가 공원 분위기.
5


In [ ]:
# predict : 학습 모델을 활용하여 다중 분류된 클래스를 출력해주는 함수
# 코드 출처 : https://hoit1302.tistory.com/159

def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    temp_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = temp_model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("주차")
            elif np.argmax(logits) == 1:
                test_eval.append("환경")
            elif np.argmax(logits) == 2:
                test_eval.append("커뮤니티")
            elif np.argmax(logits) == 3:
                test_eval.append("동별특징")
            elif np.argmax(logits) == 4:
                test_eval.append("주변상권")
            elif np.argmax(logits) == 5:
                test_eval.append("교통")
            elif np.argmax(logits) == 6:
                test_eval.append("학군")
            elif np.argmax(logits) == 7:
                test_eval.append("소음")

        print(">> 입력하신 문구는 " + test_eval[0] + "이군요")


In [ ]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 차가 3대인데도 불구하고 주차가 너무 편한데요


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 문구는 주차이군요


하고싶은 말을 입력해주세요 : 0
